<a href="https://colab.research.google.com/github/arresejo/misc/blob/main/dqn_128_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random

from keras.models import Sequential, load_model, clone_model
from keras.layers import Dense, Activation
from keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from collections import deque


In [4]:
import gym

!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 8.2 MB/s 


In [5]:
env_name = "LunarLander-v2"
env = gym.make(env_name).unwrapped

#env.spec.max_episode_steps = 3000

np.random.seed(0)

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 15 07:39:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [11]:
class Agent():
    def __init__(self, env, batch_size, gamma, alpha, epsilon, epsilon_decay, epsilon_min, update_frequency, memory_size):
        self.env = env
        if not self.env.continuous:  # discrete
            self.num_actions = 4  # line 170 in source code: https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py
        else:
            return f"you still did not work on coninuous..."
        self.memory_size = memory_size
        memory_space = 8 + 1 + 1 + 8 + 1  # states, action, reward, next_state, terminal   # TDL (soft-code it from the env)
        self.memory_counter = 0
        #self.memory = np.zeros((memory_space, self.memory_size))
        self.memory = deque(maxlen=memory_size)

        self.batch_size = batch_size
        self.gamma = gamma
        self.alpha = alpha
        # self.num_episodes = num_episodes  # in the train method
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.update_frequency = update_frequency

    def agent_model(self, layer_list, input_dims, lr, loss, optimizer):  # fcn
        model = Sequential()
        for l in layer_list:
            model.add(l)
        model.compile(optimizer=optimizer(learning_rate=lr), loss=loss)
        return model

    def store_in_memory(self, sarst_list):  # sarst_list := (state,action,next_state,reward,terminal)
        # store transition (S, A, R, S', terminal) in replay memory D
        self.memory[:, self.memory_counter % self.memory_size] = np.concatenate(
            tuple(np.array(x, dtype=float, ndmin=1) for x in sarst_list))
        self.memory_counter += 1

    def choose_action(self, state):
        if np.random.random() > self.epsilon:
            action = np.argmax(self.model_q1.predict(state.reshape((1, 8))))  # epsilon-greedy
        else:
            action = np.random.randint(self.num_actions)
        return action

    def get_rand_sample(self):
        batch_indexes = np.random.choice(len(np.array(self.memory)), self.batch_size)

        states, actions, next_states, rewards, terminals = ([] for _ in range(5))

        for index in batch_indexes:
            states.append(self.memory[index][0])
            actions.append(self.memory[index][1])
            next_states.append(self.memory[index][2])
            rewards.append(self.memory[index][3])
            terminals.append(self.memory[index][4])

        return np.array(states).squeeze(), \
               np.array(actions).squeeze(), \
               np.array(next_states), \
               np.array(rewards), \
               np.array(terminals)

    def train(self, num_episodes, layer_list, input_dims=8, loss='mse',
              optimizer=Adam):  # TDL: default of input_dims as num of states
        self.model_q1 = self.agent_model(layer_list, input_dims, self.alpha, loss, optimizer)
        #self.model_q2 = clone_model(self.model_q1)
        #self.model_q2.set_weights(self.model_q1.get_weights())
        reward_steps_hist = []  # tupple per episode:  (reward, num_steps)
        scores  = []

        for episode in range(num_episodes):
            total_reward = 0
            steps = 0
            terminal = False
            state = self.env.reset()  # seed=seed

            score = 0

            if episode != 0 and episode % 20 == 0:
              self.model_q1.save("/content/saved_models/model_"+str(episode)+"_episodes.h5")

            #while not terminal:
            for _ in range(3000):
                action = self.choose_action(state)
                next_state, reward, terminal, _ = self.env.step(action)

                self.memory.append((state, action, next_state, reward, terminal))

                total_reward += reward

                if len(self.memory) >= self.batch_size:

                    states, actions, next_states, rewards, terminals = self.get_rand_sample()

                    #q_next_state = self.model_q1.predict_on_batch(next_states)
                    #q_target = self.model_q1.predict_on_batch(states)
                    q_next_state = self.model_q1.predict(next_states)
                    q_target = self.model_q1.predict(states)

                    #q_target[np.arange(self.batch_size), actions] = rewards + (terminals == 0) * self.gamma * np.amax(q_next_state, axis=1)
                    q_target[np.arange(self.batch_size), actions] = rewards + self.gamma * np.amax(q_next_state, axis=1) *  (1 - terminals)

                    self.model_q1.fit(states, q_target, verbose=0)

                    self.epsilon = np.max(((self.epsilon * self.epsilon_decay), self.epsilon_min))

                state = next_state
                score += reward
                steps += 1

                if terminal:
                  scores.append(score)
                  print("Episode = {}, Score = {}, Avg_Score = {}".format(episode, score, np.mean(scores[-100:])))
                  print((episode, total_reward, steps, self.epsilon))
                  break

            print(f"End of episode {episode}")

            #print((episode, total_reward, steps, self.epsilon))

            reward_steps_hist.append((total_reward, steps))

In [ ]:
batch_size = 64

gamma = .99
alpha = .001

epsilon = 1
epsilon_decay = .996
epsilon_min = .01

update_frequency = 10  # in steps
memory_size = 1000000
num_episodes = 400

layers_list = [Dense(128, input_shape=(8,), activation='relu'), Dense(128, activation='relu'), Dense(4, activation='linear')]  # TDL: soft-code inut_shape and num_actions (as part of the class)

agent = Agent(env, batch_size, gamma, alpha, epsilon, epsilon_decay, epsilon_min, update_frequency, memory_size)

agent.train(num_episodes, layers_list, input_dims=8, loss='mse', optimizer=Adam)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Episode = 0, Score = -111.62410467234731, Avg_Score = -111.62410467234731
(0, -111.62410467234731, 110, 0.8283023736574049)
End of episode 0
Episode = 1, Score = -159.0263411855378, Avg_Score = -135.32522292894257
(1, -159.0263411855378, 91, 0.5751600363283949)
End of episode 1
Episode = 2, Score = -230.92644392482126, Avg_Score = -167.19229659423547
(2, -230.92644392482126, 95, 0.3930301369795873)
End of episode 2
Episode = 3, Score = 18.131013564500563, Avg_Score = -120.86146905455146
(3, 18.131013564500563, 87, 0.27732453635319726)
End of episode 3
Episode = 4, Score = -487.86209045050396, Avg_Score = -194.26159333374196
(4, -487.86209045050396, 309, 0.08037553348057996)
End of episode 4
Episode = 5, Score = -696.069777538496, Avg_Score = -277.896290701201
(5, -696.069777538496, 75, 0.05950784872013358)
End of episode 5
Episode = 6, Score = -322.0690952579678, Avg_Score = -284.2066913521677
(6, -322.0690952579678, 53, 0.04811928734513095)
End of episode 6
Episode = 7, Score = -294.5